<a href="https://colab.research.google.com/github/rochadelon/Datacamp-Roadmap/blob/main/divisor_semantico_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DivisorPDFCapitulos

Este notebook demonstra como usar a classe `DivisorPDFCapitulos` para detectar e dividir capítulos de um arquivo PDF, tanto via bookmarks (outline) quanto via padrões de texto.

## 1. Instalação de Dependências

Execute o comando abaixo para instalar o PyPDF2, se ainda não estiver instalado.

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


## 2. Importações e Definição da Classe

Aqui importamos as bibliotecas necessárias e definimos a classe `DivisorPDFCapitulos`.

In [8]:
import os
import re
from PyPDF2 import PdfReader, PdfWriter
from typing import List, Dict


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:

class DivisorPDFCapitulos:
    def __init__(self, caminho_pdf: str):
        """
        Inicializa o divisor de PDF
        Args:
            caminho_pdf (str): Caminho para o arquivo PDF
        """
        self.caminho_pdf = caminho_pdf
        self.reader = PdfReader(caminho_pdf)
        self.total_paginas = len(self.reader.pages)
        self.capitulos = []

    def detectar_capitulos_por_bookmark(self) -> List[Dict]:
        """
        Detecta capítulos usando bookmarks/outline do PDF
        Returns:
            List[Dict]: Lista de capítulos com página inicial e título
        """
        capitulos = []
        if hasattr(self.reader, 'outline') and self.reader.outline:
            for item in self.reader.outline:
                if isinstance(item, dict) and '/Title' in item:
                    try:
                        pagina = self.reader.get_destination_page_number(item) + 1
                        titulo = item['/Title']
                        capitulos.append({
                            'titulo': titulo,
                            'pagina_inicial': pagina,
                            'pagina_final': None
                        })
                    except:
                        continue

        capitulos.sort(key=lambda x: x['pagina_inicial'])
        for i in range(len(capitulos)):
            if i < len(capitulos) - 1:
                capitulos[i]['pagina_final'] = capitulos[i+1]['pagina_inicial'] - 1
            else:
                capitulos[i]['pagina_final'] = self.total_paginas
        return capitulos

    def detectar_capitulos_por_texto(self, padroes: List[str] = None) -> List[Dict]:
        """
        Detecta capítulos baseado em padrões de texto
        Args:
            padroes (List[str]): Lista de padrões regex para identificar capítulos
        Returns:
            List[Dict]: Lista de capítulos detectados
        """
        if padroes is None:
            padroes = [
                r'^\s*CAPÍTULO\s+\d+',
                r'^\s*CHAPTER\s+\d+',
                r'^\s*Cap\s*\.\s*\d+',
                r'^\s*\d+\.\s+[A-ZÁÉÍÓÚ][A-Za-záéíóúÁÉÍÓÚ\s]{10,}$',
                r'^\s*[IVX]+\.\s+[A-ZÁÉÍÓÚ]',
                r'^\s*\d+\s+-\s+[A-ZÁÉÍÓÚ]'
            ]

        capitulos = []
        for num_pagina in range(self.total_paginas):
            try:
                texto = self.reader.pages[num_pagina].extract_text()
                linhas = texto.split('\n')

                for linha in linhas[:10]:
                    if not linha.strip():
                        continue

                    for padrao in padroes:
                        if re.match(padrao, linha, re.IGNORECASE):
                            capitulos.append({
                                'titulo': linha.strip(),
                                'pagina_inicial': num_pagina + 1,
                                'pagina_final': None
                            })
                            break

                    if capitulos and capitulos[-1]['pagina_inicial'] == num_pagina + 1:
                        break
            except Exception:
                continue

        capitulos.sort(key=lambda x: x['pagina_inicial'])
        for i in range(len(capitulos)):
            if i < len(capitulos) - 1:
                capitulos[i]['pagina_final'] = capitulos[i+1]['pagina_inicial'] - 1
            else:
                capitulos[i]['pagina_final'] = self.total_paginas
        return capitulos

    def dividir_por_capitulos(self, output_dir: str = 'capitulos', usar_bookmarks: bool = True) -> List[str]:
        """
        Divide o PDF em arquivos separados por capítulo
        Args:
            output_dir (str): Diretório de saída para os arquivos
            usar_bookmarks (bool): Se True, tenta usar bookmarks primeiro
        Returns:
            List[str]: Lista de caminhos dos arquivos criados
        """
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        if usar_bookmarks:
            self.capitulos = self.detectar_capitulos_por_bookmark()
            if not self.capitulos:
                self.capitulos = self.detectar_capitulos_por_texto()
        else:
            self.capitulos = self.detectar_capitulos_por_texto()

        if not self.capitulos:
            return self.dividir_por_paginas(output_dir)

        arquivos_criados = []
        for i, cap in enumerate(self.capitulos):
            writer = PdfWriter()
            for p in range(cap['pagina_inicial'] - 1, cap['pagina_final']):
                writer.add_page(self.reader.pages[p])

            nome = re.sub(r'[<>:"/\\|?*]', '_', cap['titulo'])[:50]
            nome = f"Cap_{i+1:02d}_{nome}.pdf"
            caminho = os.path.join(output_dir, nome)

            with open(caminho, 'wb') as f:
                writer.write(f)
            arquivos_criados.append(caminho)

        return arquivos_criados

    def dividir_por_paginas(self, output_dir: str = 'paginas', paginas_por_arquivo: int = 10) -> List[str]:
        """
        Divide o PDF em arquivos com número fixo de páginas
        """
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        arquivos = []
        for inicio in range(0, self.total_paginas, paginas_por_arquivo):
            writer = PdfWriter()
            fim = min(inicio + paginas_por_arquivo, self.total_paginas)

            for p in range(inicio, fim):
                writer.add_page(self.reader.pages[p])

            nome = f"Parte_{inicio+1:03d}_{fim:03d}.pdf"
            caminho = os.path.join(output_dir, nome)

            with open(caminho, 'wb') as f:
                writer.write(f)
            arquivos.append(caminho)

        return arquivos

# Exemplo de uso - teste sem arquivo para verificar se a classe foi carregada corretamente




# 3. Exemplo de uso

In [15]:
divisor = DivisorPDFCapitulos('/content/drive/MyDrive/Projetos/Ciência da Computação/Estrutura de Dados/algoritmos-teoria-e-prc3a1tica-3ed-thomas-cormen.pdf')
divisor.detectar_capitulos_por_bookmark()
divisor.dividir_por_capitulos("/content/drive/MyDrive/Nova pasta",True)

['/content/drive/MyDrive/Nova pasta/Cap_01_Folha de Rosto.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_02_Copyright.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_03_Prefácio.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_04_Sumário.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_05_Parte I _ Fundamentos.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_06_Parte II _ Ordenação e estatísticas de ordem.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_07_Parte III _ Estruturas de dados.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_08_Parte IV _ Técnicas avançadas de projeto e análise.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_09_Parte V _ Estruturas de dados avançadas.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_10_Parte VI _ Algoritmos de grafos.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_11_Parte VII _ Tópicos selecionados.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_12_Parte VIII _ Apêndice_ Fundamentos de matemática.pdf',
 '/content/drive/MyDrive/Nova pasta/Cap_13_Bibliografia.pdf',
 '/c